# Semantic Kernel

ဤကုဒ်နမူနာတွင် [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI Framework ကို အသုံးပြု၍ အခြေခံ အေးဂျင့်တစ်ခုကို ဖန်တီးပါမည်။

ဤနမူနာ၏ ရည်ရွယ်ချက်မှာ အမျိုးမျိုးသော အေးဂျင့်ပုံစံများကို အကောင်အထည်ဖော်ရာတွင် နောက်ထပ်ကုဒ်နမူနာများတွင် အသုံးပြုမည့် လုပ်ဆောင်မှုအဆင့်များကို ပြသရန် ဖြစ်သည်။


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Client ကို ဖန်တီးခြင်း

ဤနမူနာတွင် [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ကို အသုံးပြု၍ LLM ကို အသုံးပြုမည်ဖြစ်သည်။

`ai_model_id` ကို `gpt-4o-mini` အဖြစ် သတ်မှတ်ထားသည်။ GitHub Models marketplace တွင် ရရှိနိုင်သည့် အခြားမော်ဒယ်တစ်ခုကို ပြောင်းလဲကြည့်ပြီး မတူညီသော ရလဒ်များကို ကြည့်ရှုနိုင်ပါသည်။

GitHub Models အတွက် `base_url` ကို အသုံးပြုရန်လိုအပ်သည့် `Azure Inference SDK` ကို အသုံးပြုရန်အတွက် Semantic Kernel အတွင်းရှိ `OpenAIChatCompletion` connector ကို အသုံးပြုမည်ဖြစ်သည်။ အခြားမော်ဒယ်ပံ့ပိုးသူများအတွက် Semantic Kernel ကို အသုံးပြုနိုင်ရန် [ရရှိနိုင်သော အခြား connector များ](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) လည်း ရှိပါသည်။


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## အေးဂျင့်ကို ဖန်တီးခြင်း

အောက်တွင် `TravelAgent` ဟုခေါ်သော အေးဂျင့်ကို ဖန်တီးထားပါသည်။

ဤဥပမာအတွက် ကျွန်ုပ်တို့သည် အလွန်ရိုးရှင်းသောညွှန်ကြားချက်များကို အသုံးပြုထားပါသည်။ သင်သည် ဤညွှန်ကြားချက်များကို ပြောင်းလဲပြီး အေးဂျင့်၏ တုံ့ပြန်မှုကွဲပြားမှုကို ကြည့်ရှုနိုင်ပါသည်။


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## အေးဂျင့်ကို အလုပ်လုပ်စေခြင်း

အခုတော့ `ChatHistoryAgentThread` အမျိုးအစားရှိသော thread ကို သတ်မှတ်ပြီး အေးဂျင့်ကို အလုပ်လုပ်စေနိုင်ပါပြီ။ လိုအပ်သော စနစ်မက်ဆေ့များကို အေးဂျင့်၏ invoke_stream `messages` keyword argument မှတစ်ဆင့် ပေးပို့နိုင်ပါသည်။

ဒါတွေကို သတ်မှတ်ပြီးနောက်မှာတော့ `user_inputs` ကို ဖန်တီးရမည်ဖြစ်ပြီး၊ ဒါဟာ အသုံးပြုသူက အေးဂျင့်ဆီကို ပေးပို့မယ့် အချက်အလက်များဖြစ်ပါသည်။ ဒီအခါမှာတော့ `Plan me a sunny vacation` ဆိုတဲ့ မက်ဆေ့ကို သတ်မှတ်ထားပါတယ်။

ဒီမက်ဆေ့ကို ပြောင်းလဲပြီး အေးဂျင့်က ဘယ်လို တုံ့ပြန်မလဲဆိုတာ ကြည့်ရှုနိုင်ပါတယ်။


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းစာရွက်စာတမ်းကို ၎င်း၏ မူလဘာသာစကားဖြင့် အာဏာတရားရှိသော အရင်းအမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူက ဘာသာပြန်မှုကို အသုံးပြုရန် အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွတ်များ သို့မဟုတ် အနားလွဲမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
